In [2]:
# STEP 6: JOIN Operations Showcase (FINAL ENHANCED VERSION)

import pandas as pd, re, duckdb
from google.colab import files

print("⬆️ Please upload: USE3L0712.RSK, USE3L0812.RSK, Industry_to_Sector_Mapping_CLEAN.csv")
uploaded = files.upload()


def clean_col(c: str) -> str:
    c = c.strip()
    c = c.replace('%', '_PCT')
    c = re.sub(r'[^0-9A-Za-z_]', '_', c)
    c = re.sub(r'_+', '_', c)
    return c.upper()


df2007 = pd.read_csv("USE3L0712.RSK", skiprows=1, sep=",", quotechar='"')
df2007.columns = [clean_col(c) for c in df2007.columns]

df2008 = pd.read_csv("USE3L0812.RSK", skiprows=1, sep=",", quotechar='"')
df2008.columns = [clean_col(c) for c in df2008.columns]


df_map = pd.read_csv("Industry_to_Sector_Mapping_CLEAN.csv")
df_map.columns = [clean_col(c) for c in df_map.columns]

df_map["US_E3_INDUSTRY_CODE"] = pd.to_numeric(df_map["US_E3_INDUSTRY_CODE"], errors="coerce")


con = duckdb.connect()
con.register("df2007", df2007)
con.register("df2008", df2008)
con.register("df_map", df_map)

con.execute("CREATE OR REPLACE TABLE barra2007 AS SELECT * FROM df2007")
con.execute("CREATE OR REPLACE TABLE barra2008 AS SELECT * FROM df2008")
con.execute("CREATE OR REPLACE TABLE sector_map AS SELECT * FROM df_map")


print(" INNER JOIN: Surviving Companies (2007 & 2008)")
inner_df = con.execute("""
    SELECT a.BARRID, a.TICKER, a.NAME,
           a.PRICE AS price_2007, b.PRICE AS price_2008
    FROM barra2007 a
    INNER JOIN barra2008 b
      ON a.BARRID = b.BARRID
""").df()
print("Row count (survivors):", len(inner_df))
display(inner_df.head())
inner_df.to_csv("inner_join_results.csv", index=False)


print(" LEFT JOIN: 2007 Companies (flag if disappeared in 2008)")
left_df = con.execute("""
    SELECT a.BARRID, a.TICKER, a.NAME,
           a.PRICE AS price_2007, b.PRICE AS price_2008,
           CASE WHEN b.BARRID IS NULL THEN 'DROPPED' ELSE 'SURVIVED' END AS status
    FROM barra2007 a
    LEFT JOIN barra2008 b
      ON a.BARRID = b.BARRID
""").df()
print("Row count (all 2007):", len(left_df))
display(left_df.head())
left_df.to_csv("left_join_results.csv", index=False)

dropped_df = left_df[left_df["status"] == "DROPPED"]
print("Dropped in 2008:", len(dropped_df))
dropped_df.to_csv("dropped_companies.csv", index=False)

print(" FULL OUTER JOIN: All Companies (2007 + 2008)")
full_df = con.execute("""
    SELECT a.BARRID, a.TICKER, a.NAME,
           a.PRICE AS price_2007, b.PRICE AS price_2008
    FROM barra2007 a
    LEFT JOIN barra2008 b
      ON a.BARRID = b.BARRID
    UNION
    SELECT b.BARRID, b.TICKER, b.NAME,
           a.PRICE AS price_2007, b.PRICE AS price_2008
    FROM barra2008 b
    LEFT JOIN barra2007 a
      ON a.BARRID = b.BARRID
""").df()
print("Row count (all unique firms across both years):", len(full_df))
display(full_df.head())
full_df.to_csv("full_outer_join_results.csv", index=False)


print("FULL OUTER JOIN with NULL Handling (default values)")
full_clean = con.execute("""
    SELECT COALESCE(a.BARRID, b.BARRID) AS BARRID,
           COALESCE(a.TICKER, b.TICKER) AS TICKER,
           COALESCE(a.NAME, b.NAME)     AS NAME,
           COALESCE(a.PRICE, 0)         AS price_2007,
           COALESCE(b.PRICE, 0)         AS price_2008
    FROM barra2007 a
    FULL OUTER JOIN barra2008 b
      ON a.BARRID = b.BARRID
""").df()
print("Row count (clean outer join):", len(full_clean))
display(full_clean.head())
full_clean.to_csv("full_outer_join_clean.csv", index=False)


print(" SECTOR-LEVEL JOIN: Average Beta by Sector (2007 vs 2008)")
sector_df = con.execute("""
    WITH survivors AS (
        SELECT a.BARRID, a.BETA AS BETA_2007, b.BETA AS BETA_2008, a.IND1
        FROM barra2007 a
        INNER JOIN barra2008 b
          ON a.BARRID = b.BARRID
    )
    SELECT m.SECTOR,
           ROUND(AVG(s.BETA_2007), 3) AS avg_beta_2007,
           ROUND(AVG(s.BETA_2008), 3) AS avg_beta_2008
    FROM survivors s
    INNER JOIN sector_map m
      ON s.IND1 = m.US_E3_INDUSTRY_CODE
    GROUP BY m.SECTOR
    ORDER BY m.SECTOR
""").df()
print("Sectors covered:", len(sector_df))
display(sector_df)
sector_df.to_csv("sector_level_beta.csv", index=False)


⬆️ Please upload: USE3L0712.RSK, USE3L0812.RSK, Industry_to_Sector_Mapping_CLEAN.csv


Saving Industry_to_Sector_Mapping_CLEAN.csv to Industry_to_Sector_Mapping_CLEAN (1).csv
Saving USE3L0712.RSK to USE3L0712 (1).RSK
Saving USE3L0812.RSK to USE3L0812 (1).RSK
 INNER JOIN: Surviving Companies (2007 & 2008)
Row count (survivors): 9239


,BARRID,TICKER,NAME,price_2007,price_2008
0,USA1131,IX,ORIX CORP,84.79,28.420
1,USA1141,SAOL,SAO LUIS MINING INC,0.20,0.001
2,USA1151,IXYS,IXYS CORP,8.02,8.260
3,USA1161,CDGT,CHINA DIGITAL MEDIA CORP,0.24,0.045
4,USA11B1,SWZH,SWEETSKINZ HLDGS INC,0.02,0.010


 LEFT JOIN: 2007 Companies (flag if disappeared in 2008)
Row count (all 2007): 10214


,BARRID,TICKER,NAME,price_2007,price_2008,status
0,USA1131,IX,ORIX CORP,84.79,28.420,SURVIVED
1,USA1141,SAOL,SAO LUIS MINING INC,0.20,0.001,SURVIVED
2,USA1151,IXYS,IXYS CORP,8.02,8.260,SURVIVED
3,USA1161,CDGT,CHINA DIGITAL MEDIA CORP,0.24,0.045,SURVIVED
4,USA11B1,SWZH,SWEETSKINZ HLDGS INC,0.02,0.010,SURVIVED


...

,BARRID,TICKER,NAME,price_2007,price_2008
0,USA1131,IX,ORIX CORP,84.790,28.420
1,USA1141,SAOL,SAO LUIS MINING INC,0.200,0.001
2,USA11C1,ZLNK,ZOOLINK CORP,0.002,0.040
3,USA11D1,NTRZ,NUTRACEA,1.400,0.400
4,USA11J1,ABD,ACCO BRANDS CORP,16.040,3.450


FULL OUTER JOIN with NULL Handling (default values)
Row count (clean outer join): 10600


,BARRID,TICKER,NAME,price_2007,price_2008
0,USA1131,IX,ORIX CORP,84.79,28.420
1,USA1141,SAOL,SAO LUIS MINING INC,0.20,0.001
2,USA1151,IXYS,IXYS CORP,8.02,8.260
3,USA1161,CDGT,CHINA DIGITAL MEDIA CORP,0.24,0.045
4,USA11B1,SWZH,SWEETSKINZ HLDGS INC,0.02,0.010


 SECTOR-LEVEL JOIN: Average Beta by Sector (2007 vs 2008)
Sectors covered: 13


,SECTOR,avg_beta_2007,avg_beta_2008
0,Basic Materials,1.301,1.582
1,Commercial Services,1.213,1.433
2,Consumer Cyclicals,1.368,1.596
3,Consumer Noncyclicals,0.876,1.056
4,Consumer Services,1.143,1.619
5,Energy,1.102,1.666
6,Financial,1.134,1.205
7,Health Care,0.946,1.219
8,Industrials,1.358,1.565
9,Technology,1.248,1.381
